## Notebook to keep track of tabular data (and generate bagel for now)
- Files parsed:
    - `nipoppy_manifest.csv`: Get available participants and sessions 
    - `QPN_Imaging_Codes.xlsx`: Get demographics and clinical scores 
- Currently collating and tracking following assessments:
    - UPDRS
    - MoCA
- Files generated:
    - `bagel.csv`

In [1]:
import pandas as pd
import numpy as np
from itertools import product

In [15]:
release_dir = "/home/nikhil/projects/Parkinsons/qpn/releases/"

current_release = "Sept_2023"

# Current nipoppy manifest
current_nipoppy_manifest_csv = f"{release_dir}{current_release}/tabular/manifest.csv"

# Current demographics CSV
demographics_csv = f"{release_dir}/{current_release}/tabular/demographics/demographics.csv"

# Current PD-assessment CSV
clinical_assessment_xls = f"{release_dir}/{current_release}/tabular/Suivi_RPQ.xlsx"

# NeuroBagel CSVs
clinical_scores_csv = f"{release_dir}/{current_release}/tabular/assessments/clinical_scores_summary.csv"
bagel_tsv = f"{release_dir}/{current_release}/tabular/bagel.tsv"
dash_csv = f"{release_dir}/{current_release}/tabular/dash.csv"


### Read nipoppy manifest


In [16]:
visit_list = ["MRI_v1"] 
session_id_list = ["01"]
session_list = [f"ses-{idx}" for idx in session_id_list]

manifest_cols = ["participant_id", "visit", "session"]

current_nipoppy_manifest_df = pd.read_csv(current_nipoppy_manifest_csv)
current_nipoppy_manifest_df = current_nipoppy_manifest_df[current_nipoppy_manifest_df["visit"].isin(visit_list)]
current_nipoppy_manifest_df = current_nipoppy_manifest_df[current_nipoppy_manifest_df["session"].isin(session_list)]
current_nipoppy_manifest_df = current_nipoppy_manifest_df[manifest_cols]
nipoppy_participants = current_nipoppy_manifest_df["participant_id"].unique()
n_participants = len(nipoppy_participants)
print(f"n_participants: {n_participants}")
current_nipoppy_manifest_df.head()

n_participants: 265


,participant_id,visit,session
0,PD00016,MRI_v1,ses-01
1,PD00020,MRI_v1,ses-01
2,PD00032,MRI_v1,ses-01
3,PD00048,MRI_v1,ses-01
4,PD00119,MRI_v1,ses-01


### Read QPN_Imaging_Codes
    - Calculate age per visit (in floats)

In [ ]:
col_range = "A:N"

col_rename_dict = {"Unnamed: 0":"participant_id",
                "IRM01\n(J-M-A)":"IRM01_date", "#IRM 1\n PD":"IRM01_PD", "#IRM 1\n CTRL":"IRM01_CTRL", "# IRM 1\n RBD":"IRM01_RBD",
                "IRM 2 \n(J-M-A)":"IRM02_date", "#IRM 2\n PD":"IRM02_PD", "#IRM 2\n CTRL":"IRM02_CTRL", "# IRM 2\n RBD":"IRM02_RBD"}

useful_cols = col_rename_dict.values()

suivi_df = pd.read_excel(current_recruit_manifest_xls,sheet_name="En cours", engine='openpyxl', usecols=col_range)
suivi_df = suivi_df.rename(columns=col_rename_dict)[useful_cols].copy()

# remove the row with tally
suivi_df = suivi_df.drop([0])

# remove rows without participant_id
suivi_df = suivi_df.dropna(axis=0, subset=["participant_id"])
suivi_df = suivi_df[~suivi_df["participant_id"].astype(str).isin(["0"])] 

# remove subjects without imaging data
suivi_df = suivi_df[(suivi_df["IRM01_PD"] == 1) | (suivi_df["IRM01_CTRL"] == 1) | (suivi_df["IRM01_RBD"] == 1) | 
                    (suivi_df["IRM02_PD"] == 1) | (suivi_df["IRM02_CTRL"] == 1) | (suivi_df["IRM02_RBD"] == 1)]


# fix participant_id formatting issues
# Some rows have Dx in participant_id and one participant with two IDs with "="
possible_delimiters = [" ", "(", "="]
for delim in possible_delimiters:
    suivi_df["participant_id"] = suivi_df["participant_id"].str.split(delim, 1, expand=True)[0]

# nipoppy_participants_current
nipoppy_participants_current = suivi_df["participant_id"].dropna().unique()

suivi_df

In [45]:
df = pd.read_excel(clinical_assessment_xls,sheet_name="En cours", engine='openpyxl', nrows=2)
df.columns


Index(['Unnamed: 0', 'Unnamed: 1', 'IRM01\n(J-M-A)', '#IRM 1\n PD',
       '#IRM 1\n CTRL', '# IRM 1\n RBD', 'IRM 2 \n(J-M-A)', '#IRM 2\n PD',
       '#IRM 2\n CTRL', '# IRM 2\n RBD', 'IRM 3\n(J-M-A)', '#IRM 3\n PD',
       '#IRM 3\n CTRL', '# IRM 3\n RBD', 'MEG01 (J-M-A)', '#MEG\n PD',
       '#MEG\n CTRL', '# MEG\n RBD', 'Neuropsy01\n(J-M-A)', '#Neuropsy\n PD',
       '#Neuropsy \nCTRL', '#Neuropsy \nRBD', 'Neuropsy02\n(J-M-A)',
       '#Neuropsy 2\n PD', '#Neuropsy 2\nCTRL', '#Neuropsy 2\nRBD',
       'Neuropsy 3\n(J-M-A)', '#Neuropsy 3\n PD', '#Neuropsy 3\nCTRL',
       '#Neuropsy 3\nRBD', 'MDS-UPDRS_partie_III_1\n(J-M-A)',
       '# MDS-UPDRS part 3\n PD', '# MDS-UPDRS part 3\nRBD',
       '# MDS-UPDRS p.III 2\n(J-M-A)', '#MDS-UPDRS III\n 2 PD',
       '# MDS-UPDRS III\n2 RBD', 'MDS-UPDRS part III 3\n(J-M-A)',
       '# MDS-UPDRS III\n3 PD', '# MDS-UPDRS III\n3 RBD',
       'MDS-UPDRS_complet (J-M-A)', '# MDS-UPDRS\n PD', '# MDS-UPDRS \nRBD',
       'MDS-UPDRS\ncomplet 2', '# MDS-

In [41]:
col_range = "A:BZ"

col_rename_dict = {"Unnamed: 0":"participant_id",
                    'Neuropsy01\n(J-M-A)': 'Neuropsy01_date', 
                    '#Neuropsy\n PD': 'Neuropsy01_PD',
                    '#Neuropsy \nCTRL': 'Neuropsy01_CTRL', 
                    '#Neuropsy \nRBD': 'Neuropsy01_RBD', 
                    'Neuropsy02\n(J-M-A)': 'Neuropsy02_date',
                    '#Neuropsy 2\n PD': 'Neuropsy02_PD', 
                    '#Neuropsy 2\nCTRL': 'Neuropsy02_CTRL', 
                    '#Neuropsy 2\nRBD': 'Neuropsy02_RBD',
                    'Neuropsy 3\n(J-M-A)': 'Neuropsy03_date', 
                    '#Neuropsy 3\n PD': 'Neuropsy03_PD', 
                    '#Neuropsy 3\nCTRL': 'Neuropsy03_CTRL',
                    '#Neuropsy 3\nRBD': 'Neuropsy03_RBD', 
                    'MDS-UPDRS_partie_III_1\n(J-M-A)': 'MDS-UPDRS-III_01_date',
                    '# MDS-UPDRS part 3\n PD': 'MDS-UPDRS-III_01_PD',
                    '# MDS-UPDRS part 3\nRBD': 'MDS-UPDRS-III_01_RBD',
                    '# MDS-UPDRS p.III 2\n(J-M-A)': 'MDS-UPDRS-III_02_date', 
                    '#MDS-UPDRS III\n 2 PD': 'MDS-UPDRS-III_02_PD',
                    '# MDS-UPDRS III\n2 RBD': 'MDS-UPDRS-III_02_RBD', 
                    'MDS-UPDRS part III 3\n(J-M-A)': 'MDS-UPDRS-III_03_date',
                    '# MDS-UPDRS III\n3 PD': 'MDS-UPDRS-III_03_PD', 
                    '# MDS-UPDRS III\n3 RBD': 'MDS-UPDRS-III_03_RBD',
                    'MDS-UPDRS_complet (J-M-A)': 'MDS-UPDRS_complet_01_date', 
                    '# MDS-UPDRS\n PD': 'MDS-UPDRS_complet_01_PD', 
                    '# MDS-UPDRS \nRBD': 'MDS-UPDRS_complet_01_RBD',
                    'MDS-UPDRS\ncomplet 2': 'MDS-UPDRS_complet_02_date', 
                    '# MDS-UPDRS 2\n PD': 'MDS-UPDRS_complet_02_PD', 
                    '# MDS-UPDRS 2\nRBD': 'MDS-UPDRS_complet_02_RBD',
                    'MDS-UPDRS complet 3\n(J-M-A)': 'MDS-UPDRS_complet_03_date',
                    '# MDS-UPDRS\n 3 PD': 'MDS-UPDRS_complet_03_PD',
                    '# MDS-UPDRS\n3 RBD': 'MDS-UPDRS_complet_03_RBD', 
                    'MoCA': 'MoCA_01_date', 
                    '#MoCA\n PD': 'MoCA_01_PD',
                    '#MoCA\nCTRL': 'MoCA_01_CTRL', 
                    '#MoCA\nRBD': 'MoCA_01_RBD',
                    'MoCA 2': 'MoCA_02', 
                    '#MoCA 2\n PD': 'MoCA_02_PD', 
                    '#MoCA 2\nCTRL': 'MoCA_02_CTRL', 
                    '#MoCA 2\nRBD': 'MoCA_02_RBD', 
                    'MoCA 3': 'MoCA_03',
                    '#MoCA 3\n PD': 'MoCA_03_PD', 
                    '#MoCA 3\nCTRL': 'MoCA_03_CTRL', 
                    '#MoCA 3\nRBD': 'MoCA_03_RBD',
                    'Speech Sample (J-M-A)': 'Speech_Sample_01_date', 
                    '#speech sample PD': 'Speech_Sample_01_PD', 
                    '#speech sample CTRL': 'Speech_Sample_01_CTRL',
                    '#speech sample RBD': 'Speech_Sample_01_RBD', 
                    'Neurocognitive_1\n(J-M-A)': 'Neurocognitive_01_date', 
                    '#Neurocognitive 1 \n PD': 'Neurocognitive_01_PD',
                    '#Neurocognitive 1 \nCTRL': 'Neurocognitive_01_CTRL', 
                    'Neurocognitive_2\n(J-M-A)': 'Neurocognitive_02_date',
                    '#Neurocognitive 2\n PD': 'Neurocognitive_02_PD', 
                    '#Neurocognitive 2\nCTRL': 'Neurocognitive_02_CTRL',
                    }

useful_cols = col_rename_dict.values()

suivi_df = pd.read_excel(clinical_assessment_xls,sheet_name="En cours", engine='openpyxl', usecols=col_range)
suivi_df = suivi_df.rename(columns=col_rename_dict)
suivi_df = suivi_df[useful_cols].copy()


# remove the row with tally
suivi_df = suivi_df.drop([0])

# remove rows without participant_id
suivi_df = suivi_df.dropna(axis=0, subset=["participant_id"])
suivi_df = suivi_df[~suivi_df["participant_id"].astype(str).isin(["0"])] 
suivi_df

,participant_id,Neuropsy01_date,Neuropsy01_PD,Neuropsy01_CTRL,Neuropsy01_RBD,Neuropsy02_date,Neuropsy02_PD,Neuropsy02_CTRL,Neuropsy02_RBD,Neuropsy03_date,...,Speech_Sample_01_date,Speech_Sample_01_PD,Speech_Sample_01_CTRL,Speech_Sample_01_RBD,Neurocognitive_01_date,Neurocognitive_01_PD,Neurocognitive_01_CTRL,Neurocognitive_02_date,Neurocognitive_02_PD,Neurocognitive_02_CTRL
1,PD00002,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,PD00003,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,PD00004,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,PD00005,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
5,PD00006,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,MNI0552,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2189,MNI0553,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2190,MNI0554,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2191,MNI0555,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [10]:
demo_cols = ["participant_id", "sex", "dob", "group", "visit_01"]
demo_df =  pd.read_excel(clinical_assessment_xls,sheet_name="recruit_manifest", engine='openpyxl').dropna(how="all", axis=0)
demo_df = demo_df[demo_cols]
demo_participants = demo_df["participant_id"].unique()
n_demo_participants = len(demo_participants)
print(f"n_demo_participants: {n_demo_participants}")

current_nipoppy_manifest_df = pd.merge(current_nipoppy_manifest_df, demo_df, on=["participant_id"], how="left")

visit_date_col = "visit_01"

# Parse and format dates: doing it in two steps because simple "format="%d-%m-%Y" skips MRI02 dates
current_nipoppy_manifest_df["dob"] = pd.to_datetime(current_nipoppy_manifest_df["dob"], errors="coerce", dayfirst=True)
current_nipoppy_manifest_df["dob"] = pd.to_datetime(current_nipoppy_manifest_df["dob"], errors="coerce", format="%d-%m-%Y")

current_nipoppy_manifest_df[visit_date_col] = pd.to_datetime(current_nipoppy_manifest_df[visit_date_col], errors="coerce", dayfirst=True)
current_nipoppy_manifest_df[visit_date_col] = pd.to_datetime(current_nipoppy_manifest_df[visit_date_col], errors="coerce", format="%d-%m-%Y")

current_nipoppy_manifest_df[f"age"] = (current_nipoppy_manifest_df[visit_date_col] - current_nipoppy_manifest_df["dob"])
current_nipoppy_manifest_df[f"age"] = np.round(current_nipoppy_manifest_df[f"age"] / np.timedelta64(1, 'Y'), 1)
 
current_nipoppy_manifest_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/nikhil/projects/Parkinsons/qpn/releases//Sept_2023/tabular/demographics/Suivi_RPQ.xlsx'

### Save demographics to CSV

In [34]:
save_data = False

save_demo_cols = ["participant_id", "visit", "session", "age", "sex", "dob", "group", "date_of_visit"]
save_demo_df = current_nipoppy_manifest_df.rename(columns={"visit_01": "date_of_visit"})
save_demo_df = save_demo_df[save_demo_cols]

if save_data:   
    save_demo_df.to_csv(demographics_csv, index=False)

save_demo_df.head()

,participant_id,visit,session,age,sex,dob,group,date_of_visit
0,MNI0056,V01,ses-01,79.2,M,1942-05-21,PD,2021-08-18
1,MNI0058,V01,ses-01,57.4,M,1964-03-14,PD,2021-08-18
2,MNI0068,V01,ses-01,69.1,M,1952-08-05,PD,2021-08-27
3,MNI0079,V01,ses-01,50.1,F,1971-11-25,PD,2021-12-22
4,MNI0103,V01,ses-01,82.0,M,1939-08-17,PD,2021-08-11


### Read UPDRS and MoCA for imaging subjects

In [35]:
MoCA_df =  pd.read_excel(clinical_assessment_xls,sheet_name="MoCA", engine='openpyxl').dropna(how="all", axis=0)
MoCA_df["moca_date"] = pd.to_datetime(MoCA_df["moca_date"])
print(f"MoCA df shape: {MoCA_df.shape}")
print("Droping participants without a MoCA date")
MoCA_df = MoCA_df[~MoCA_df["moca_date"].isna()]
print(f"MoCA df shape: {MoCA_df.shape}")

participants_multiple_visits = MoCA_df[MoCA_df.duplicated(subset=["participant_id"])]["participant_id"].unique()
print(f"Number of participants with multiple visits: {len(participants_multiple_visits)}")
print("Dropping second or third visits for now - need to define visits based on the time interval between visits")
MoCA_df = MoCA_df.sort_values(by=["participant_id","moca_date"]).drop_duplicates(subset=["participant_id"],keep="first")

print(f"MoCA df shape: {MoCA_df.shape}")
MoCA_df.head()

MoCA df shape: (232, 26)
Droping participants without a MoCA date
MoCA df shape: (179, 26)
Number of participants with multiple visits: 27
Dropping second or third visits for now - need to define visits based on the time interval between visits
MoCA df shape: (151, 26)


,participant_id,delete,delete.1,moca_date,moca_vse,moca_nam,moca_att,moca_lang,moca_abs,moca_del,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,MNI0056,NaN,T1,2021-11-06,2.0,3.0,4.0,1.0,2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MNI0058,NaN,T1,2021-07-23,4.0,3.0,5.0,2.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MNI0079,NaN,T2,2021-12-22,2.0,3.0,5.0,3.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MNI0103,NaN,T1,2021-08-11,1.0,3.0,5.0,2.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MNI0109,NaN,T1,2021-11-24,5.0,3.0,6.0,3.0,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Identify duplicate participants (i.e. multiple visits)

In [36]:
UPDRS_df =  pd.read_excel(clinical_assessment_xls,sheet_name="MDS-UPDRS", engine='openpyxl').dropna(how="all", axis=0)
UPDRS_df = UPDRS_df.drop(0) # column name description gets assigned to first row for some reason

UPDRS_df["updrs_date"] = pd.to_datetime(UPDRS_df["updrs_date"])
print(f"UPDRS df shape: {UPDRS_df.shape}")
print("Droping participants without a UPDRS date")
UPDRS_df = UPDRS_df[~UPDRS_df["updrs_date"].isna()]
print(f"UPDRS df shape: {UPDRS_df.shape}")

participants_multiple_visits = UPDRS_df[UPDRS_df.duplicated(subset=["participant_id"])]["participant_id"].unique()
print(f"Number of participants with multiple visits: {len(participants_multiple_visits)}")

print("Dropping second or third visits for now - need to define visits based on the time interval between visits")
UPDRS_df = UPDRS_df.sort_values(by=["participant_id","updrs_date"]).drop_duplicates(subset=["participant_id"],keep="first")

print(f"UPDRS df shape: {UPDRS_df.shape}")

UPDRS_df.head()

UPDRS df shape: (208, 84)
Droping participants without a UPDRS date
UPDRS df shape: (94, 84)
Number of participants with multiple visits: 0
Dropping second or third visits for now - need to define visits based on the time interval between visits
UPDRS df shape: (94, 84)


,participant_id,updrs_date,updrs_staff,updrs_in_person,updrs_1_a,updrs_1_1,updrs_1_2,updrs_1_3,updrs_1_4,updrs_1_5,...,updrs_4_1,updrs_4_2,updrs_4_3,updrs_4_4,updrs_4_5,updrs_4_6,updrs_1_total,updrs_2_total,updrs_3_total,updrs_4_total
1,MNI0056,2021-11-06,Sarah Bogard,2,1,0,0,0,1,0,...,0,0,0,0,0,0,12,22,58,0
2,MNI0058,2021-07-23,Sarah Bogard,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,6,26,0
3,MNI0068,2021-08-27,Sarah Bogard,2,3,2,0,1,1,0,...,0,0,0,0,0,0,11,8,28,0
4,MNI0079,2022-01-21,Roozbeh Sattari,2,1,0,0,0,1,0,...,1,1,1,2,1,2,13,10,22,8
5,MNI0103,2021-01-10,Sarah Bogard,2,1,1,2,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,28,25,43,NaN


### Merge assessments into NeuroBagel format
- Manifest: [`participant_id`, `bids_id`, `session_id`, `age`, `sex`, `dob`, `group`]
- MoCA column(s): [`moca_total`]
- UPDRS column(s): [`updrs_1_total`,`updrs_2_total`,`updrs_3_total`,`updrs_4_total`]

In [37]:
manifest_cols = ["bids_id", "age", "sex", "dob", "group"]
moca_cols = ["moca_total"]
updrs_cols = ["updrs_1_total","updrs_2_total","updrs_3_total","updrs_4_total"]

participant_ids = current_nipoppy_manifest_df["participant_id"].values
a,b = zip(*list(product(participant_ids,session_id_list)))

bagel_df = pd.DataFrame()
bagel_df["participant_id"] = a
bagel_df["session_id"] = b
bagel_df["session_id"] = bagel_df["session_id"].astype(str)

bagel_df = pd.merge(bagel_df,current_nipoppy_manifest_df[["participant_id"] + manifest_cols],
                    on="participant_id",how="left")

bagel_df = pd.merge(bagel_df,MoCA_df[["participant_id"] + moca_cols],
                    on="participant_id",how="left")
                    
bagel_df = pd.merge(bagel_df,UPDRS_df[["participant_id"] + updrs_cols],
                      on="participant_id",how="left")

print(f"bagel df shape: {bagel_df.shape}")
bagel_df.head()

bagel df shape: (208, 12)


,participant_id,session_id,bids_id,age,sex,dob,group,moca_total,updrs_1_total,updrs_2_total,updrs_3_total,updrs_4_total
0,MNI0056,01,sub-MNI0056,79.2,M,1942-05-21,PD,21.0,12,22,58,0
1,MNI0058,01,sub-MNI0058,57.4,M,1964-03-14,PD,24.0,0,6,26,0
2,MNI0068,01,sub-MNI0068,69.1,M,1952-08-05,PD,NaN,11,8,28,0
3,MNI0079,01,sub-MNI0079,50.1,F,1971-11-25,PD,26.0,13,10,22,8
4,MNI0103,01,sub-MNI0103,82.0,M,1939-08-17,PD,20.0,28,25,43,NaN


### Assign availability status per assessment column

In [38]:
print(f"n_nipoppy_participants: {n_participants}")

for col in moca_cols + updrs_cols:
    bagel_df[f"{col}_status"] = ~bagel_df[col].isna()
    n_available_participants = np.sum(bagel_df[f"{col}_status"])
    print(f"{col}, n_available_participants: {n_available_participants}")

print(f"bagel df shape: {bagel_df.shape}")
bagel_df.head()

n_mr_proc_participants: 208
moca_total, n_available_participants: 151
updrs_1_total, n_available_participants: 90
updrs_2_total, n_available_participants: 90
updrs_3_total, n_available_participants: 93
updrs_4_total, n_available_participants: 89
bagel df shape: (208, 17)


,participant_id,session_id,bids_id,age,sex,dob,group,moca_total,updrs_1_total,updrs_2_total,updrs_3_total,updrs_4_total,moca_total_status,updrs_1_total_status,updrs_2_total_status,updrs_3_total_status,updrs_4_total_status
0,MNI0056,01,sub-MNI0056,79.2,M,1942-05-21,PD,21.0,12,22,58,0,True,True,True,True,True
1,MNI0058,01,sub-MNI0058,57.4,M,1964-03-14,PD,24.0,0,6,26,0,True,True,True,True,True
2,MNI0068,01,sub-MNI0068,69.1,M,1952-08-05,PD,NaN,11,8,28,0,False,True,True,True,True
3,MNI0079,01,sub-MNI0079,50.1,F,1971-11-25,PD,26.0,13,10,22,8,True,True,True,True,True
4,MNI0103,01,sub-MNI0103,82.0,M,1939-08-17,PD,20.0,28,25,43,NaN,True,True,True,True,False


### Save bagel_df

In [39]:
save_bagel = False

print(f"Save path: {bagel_tsv}")
if save_bagel:
    bagel_df.to_csv(clinical_scores_csv, sep="\t")
    bagel_df.to_csv(bagel_tsv, sep="\t")

Save path: /home/nikhil/projects/Parkinsons/qpn/releases//Apr_2023/tabular/bagel.tsv


### Create dashboard bagel

In [40]:
bagel_df["dob"] = bagel_df["dob"].astype(str).str.split("-").str[0]
# bagel_df["dob"] = bagel_df["year_of_birth"].copy()
bagel_df = bagel_df.rename(columns={"dob": "year_of_birth"})

dash_df = bagel_df.melt(id_vars=["bids_id", "participant_id", "session_id"], 
                        var_name='assessment_name', value_name='assessment_score')

dash_df = dash_df.rename(columns={"session_id": "session"})
dash_df.head()

,bids_id,participant_id,session,assessment_name,assessment_score
0,sub-MNI0056,MNI0056,01,age,79.2
1,sub-MNI0058,MNI0058,01,age,57.4
2,sub-MNI0068,MNI0068,01,age,69.1
3,sub-MNI0079,MNI0079,01,age,50.1
4,sub-MNI0103,MNI0103,01,age,82


In [41]:
# save to csv
# dash_df.to_csv(dash_csv, index=False)

### Compare manifest and bagel

In [ ]:
release_dir = f"/home/nikhil/projects/Parkinsons/qpn/releases/{current_release}/"

# Current nipoppy manifest
current_nipoppy_manifest_csv = f"{release_dir}/tabular/nipoppy_manifest.csv"
manifest_df = pd.read_csv(current_nipoppy_manifest_csv)

# NeuroBagel CSVs
bagel_tsv = f"{release_dir}/tabular/bagel.tsv"
bagel_df = pd.read_csv(bagel_tsv, sep="\t")

# bids dir list
bids_dir_id_txt = f"{release_dir}/tabular/bids_dir_subject_list.txt"
bids_dir_id_list = pd.read_csv(bids_dir_id_txt, header=None)[0].values

In [ ]:
manifest_participant_ids = set(manifest_df["participant_id"].unique())
bagel_participant_ids = set(bagel_df["participant_id"].unique())
participant_id_match = manifest_participant_ids == bagel_participant_ids

print(f"n_manifest_participants: {len(manifest_participant_ids)}, n_bagel_participants: {len(bagel_participant_ids)}")
print(f"participant_id_match: {participant_id_match}")

manifest_bids_ids = set(manifest_df["bids_id"].dropna().unique())
bagel_bids_ids = set(bagel_df["bids_id"].dropna().unique())
bids_id_match = manifest_bids_ids == bagel_bids_ids
bids_dir_id_match = bagel_bids_ids == set(bids_dir_id_list)

print(f"n_manifest_participants: {len(manifest_bids_ids)}, n_bagel_participants: {len(manifest_bids_ids)}")
print(f"bids_id_match: {bids_id_match}, bids_dir_id_match: {bids_dir_id_match}")

